
# Module 4: AI-Powered Marketing Personalization & Prediction

## Learning Objectives
- Build type-safe personalization systems
- Implement predictive analytics with error tracking
- Create robust customer segmentation
- Develop production-ready A/B testing
- Design conversion prediction models

## Prerequisites
- Modules 1-3 completion
- Python type hinting knowledge
- Basic error handling understanding
- API access for your platforms



## 1. Environment Setup

Install required packages and configure logging.


In [ ]:

!pip install swarms python-dotenv pandas numpy scikit-learn loguru


In [ ]:

import os
from typing import Dict, List, Optional, Any, Tuple, Union
from datetime import datetime, timedelta
from loguru import logger
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from dataclasses import dataclass
from dotenv import load_dotenv
from swarms import Agent, MixtureOfAgents
from swarm_models import OpenAIChat

# Configure logger
logger.add("marketing_ai.log", rotation="500 MB")

# Load environment
load_dotenv()

# Initialize model
model = OpenAIChat(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4",
    temperature=0.7
)

@dataclass
class MarketingMetrics:
    conversion_rate: float
    engagement_rate: float
    roi: float
    timestamp: datetime

@dataclass
class UserSegment:
    segment_id: int
    behavior_score: float
    value_score: float
    engagement_level: str
    characteristics: Dict[str, Any]



## 2. Type-Safe Content Personalization System

Create a robust personalization system with proper error handling.


In [ ]:

class ContentPersonalizer:
    def __init__(self, model: Any) -> None:
        self.model = model
        self.agent = Agent(
            agent_name="Personalization-Expert",
            system_prompt="""
            You are an AI Marketing Personalization Expert. Create personalized content 
            based on user segments and behaviors. Return ONLY JSON in this format:
            {
                "segment_name": str,
                "personalized_content": {
                    "headline": str,
                    "main_message": str,
                    "call_to_action": str,
                    "tone": str,
                    "key_benefits": list
                },
                "channel_adaptations": {
                    "email": {content details},
                    "social": {content details},
                    "web": {content details}
                },
                "predicted_engagement_score": float
            }
            """,
            llm=model,
            output_type="json"
        )
    
    def create_user_segment(self, user_data: pd.DataFrame) -> Dict[int, UserSegment]:
        try:
            scaler = StandardScaler()
            features_scaled = scaler.fit_transform(user_data)
            
            kmeans = KMeans(n_clusters=5, random_state=42)
            segments = kmeans.fit_predict(features_scaled)
            
            user_data['segment'] = segments
            segment_profiles = user_data.groupby('segment').mean()
            
            user_segments: Dict[int, UserSegment] = {}
            for segment_id in range(len(segment_profiles)):
                profile = segment_profiles.loc[segment_id]
                user_segments[segment_id] = UserSegment(
                    segment_id=segment_id,
                    behavior_score=float(profile['engagement_rate']),
                    value_score=float(profile['avg_order_value']),
                    engagement_level=self._calculate_engagement_level(profile),
                    characteristics=profile.to_dict()
                )
            
            return user_segments
        
        except Exception as e:
            logger.error(f"Error in create_user_segment: {str(e)}")
            raise
    
    def _calculate_engagement_level(self, profile: pd.Series) -> str:
        engagement_score = float(profile['engagement_rate'])
        if engagement_score > 0.7:
            return "high"
        elif engagement_score > 0.3:
            return "medium"
        return "low"
    
    def generate_personalized_content(self, segment: UserSegment) -> Dict[str, Any]:
        try:
            response = self.agent.run(json.dumps(segment.__dict__))
            return json.loads(response)
        except Exception as e:
            logger.error(f"Error generating content for segment {segment.segment_id}: {str(e)}")
            raise

# Example usage with type hints
def create_sample_data() -> pd.DataFrame:
    return pd.DataFrame({
        'engagement_rate': np.random.uniform(0, 1, 1000),
        'purchase_frequency': np.random.uniform(0, 10, 1000),
        'avg_order_value': np.random.uniform(50, 500, 1000),
        'email_open_rate': np.random.uniform(0, 1, 1000),
        'website_visits': np.random.uniform(1, 100, 1000)
    })

# Initialize and run
personalizer = ContentPersonalizer(model)
sample_data = create_sample_data()

try:
    segments = personalizer.create_user_segment(sample_data)
    content = personalizer.generate_personalized_content(
        next(iter(segments.values()))
    )
    print(json.dumps(content, indent=2))
except Exception as e:
    logger.error(f"Error in personalization workflow: {str(e)}")



## 3. Type-Safe Predictive Analytics

Build a robust prediction system with proper error handling and type safety.


In [ ]:

@dataclass
class CampaignPrediction:
    conversion_rate: float
    engagement_rate: float
    roi: float
    confidence_score: float
    contributing_factors: List[Dict[str, Any]]

class PredictiveAnalytics:
    def __init__(self, model: Any) -> None:
        self.model = model
        self.prediction_agent = Agent(
            agent_name="Prediction-Expert",
            system_prompt="""
            You are an AI Marketing Prediction Expert. Analyze marketing data and 
            predict outcomes. Return ONLY JSON in this format:
            {
                "predictions": {
                    "conversion_rate": float,
                    "engagement_rate": float,
                    "roi": float,
                    "confidence_score": float
                },
                "contributing_factors": [
                    {
                        "factor": str,
                        "impact_score": float,
                        "recommendation": str
                    }
                ]
            }
            """,
            llm=model,
            output_type="json"
        )
    
    def analyze_historical_data(
        self, 
        data: pd.DataFrame
    ) -> Dict[str, MarketingMetrics]:
        try:
            metrics_by_month: Dict[str, MarketingMetrics] = {}
            
            for month, group in data.groupby(data.index.to_period('M')):
                metrics_by_month[str(month)] = MarketingMetrics(
                    conversion_rate=float(group['conversion_rate'].mean()),
                    engagement_rate=float(group['engagement_rate'].mean()),
                    roi=float(group['roi'].mean()),
                    timestamp=group.index[0].to_pydatetime()
                )
            
            return metrics_by_month
            
        except Exception as e:
            logger.error(f"Error analyzing historical data: {str(e)}")
            raise
    
    def predict_campaign_performance(
        self,
        historical_metrics: Dict[str, MarketingMetrics],
        campaign_data: Dict[str, Any]
    ) -> CampaignPrediction:
        try:
            analysis_data = {
                'historical_metrics': {
                    k: v.__dict__ for k, v in historical_metrics.items()
                },
                'campaign_details': campaign_data
            }
            
            prediction = json.loads(
                self.prediction_agent.run(json.dumps(analysis_data))
            )
            
            return CampaignPrediction(
                conversion_rate=prediction['predictions']['conversion_rate'],
                engagement_rate=prediction['predictions']['engagement_rate'],
                roi=prediction['predictions']['roi'],
                confidence_score=prediction['predictions']['confidence_score'],
                contributing_factors=prediction['contributing_factors']
            )
            
        except Exception as e:
            logger.error(f"Error predicting campaign performance: {str(e)}")
            raise

# Example usage
def create_historical_data() -> pd.DataFrame:
    dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='D')
    return pd.DataFrame({
        'conversion_rate': np.random.normal(0.15, 0.03, len(dates)),
        'engagement_rate': np.random.normal(0.25, 0.05, len(dates)),
        'roi': np.random.normal(2.5, 0.5, len(dates))
    }, index=dates)

predictor = PredictiveAnalytics(model)
historical_data = create_historical_data()

try:
    historical_metrics = predictor.analyze_historical_data(historical_data)
    
    campaign_data = {
        "type": "product_launch",
        "budget": 50000,
        "duration_days": 30,
        "channels": ["email", "social", "ppc"],
        "target_audience": "tech_professionals"
    }
    
    prediction = predictor.predict_campaign_performance(
        historical_metrics,
        campaign_data
    )
    
    print(f"Campaign Prediction:")
    print(f"Conversion Rate: {prediction.conversion_rate:.2%}")
    print(f"ROI: {prediction.roi:.2f}x")
    print("Contributing Factors:")
    for factor in prediction.contributing_factors:
        print(f"- {factor['factor']}: Impact {factor['impact_score']:.2f}")
        
except Exception as e:
    logger.error(f"Error in prediction workflow: {str(e)}")


In [ ]:

def _generate_recommendations(
        self,
        platform_metrics: Dict[str, PlatformMetrics]
    ) -> List[str]:
        recommendations = []
        
        for platform, metrics in platform_metrics.items():
            # Check engagement rates
            if metrics.engagement_rate < 0.05:
                recommendations.append(
                    f"Increase {platform} engagement by optimizing posting times and content format"
                )
            
            # Check reach metrics
            if metrics.reach < 1000:  # Customize threshold based on goals
                recommendations.append(
                    f"Expand {platform} reach through targeted promotion and hashtag optimization"
                )
            
            # Platform-specific recommendations
            if platform == "linkedin":
                if metrics.raw_metrics.get('share_count', 0) < 10:
                    recommendations.append(
                        "Enhance LinkedIn shareability by including industry insights and statistics"
                    )
                    
            elif platform == "instagram":
                if metrics.raw_metrics.get('saves', 0) < 50:
                    recommendations.append(
                        "Improve Instagram save rate by creating more valuable, saveable content"
                    )
                    
            elif platform == "youtube":
                if float(metrics.raw_metrics.get('averageViewDuration', 0)) < 60:
                    recommendations.append(
                        "Increase YouTube watch time by optimizing first 30 seconds of videos"
                    )
        
        return recommendations

# Example usage of analytics
analytics = CrossPlatformAnalytics(platform_manager)

# Sample content IDs from previous publications
content_ids = {
    'linkedin': 'your_linkedin_post_id',
    'instagram': 'your_instagram_post_id',
    'youtube': 'your_youtube_video_id'
}

try:
    report = analytics.generate_report(
        start_date=datetime.now() - timedelta(days=7),
        end_date=datetime.now(),
        content_ids=content_ids
    )
    
    print("Analytics Report:")
    print(f"Total Reach: {report.total_reach:,}")
    print(f"Total Engagement: {report.total_engagement:,}")
    print("Top Performing Content:")
    for content in report.top_performing_content:
        print(f"- {content['platform']}: {content['engagement_rate']:.2%} engagement")
    print("Recommendations:")
    for rec in report.recommendations:
        print(f"- {rec}")
        
except Exception as e:
    logger.error(f"Analytics reporting error: {str(e)}")



## Automated Content Optimization System

Create a system that automatically optimizes content based on performance data.


In [ ]:

@dataclass
class OptimizationSuggestion:
    platform: str
    content_type: str
    changes: List[str]
    expected_impact: float
    implementation_priority: str

class ContentOptimizer:
    def __init__(self, platform_manager: PlatformManager) -> None:
        self.platform_manager = platform_manager
        self.optimization_agent = Agent(
            agent_name="Content-Optimization-Expert",
            system_prompt="""
            You are a Content Optimization Expert. Analyze performance data and suggest
            improvements. Return ONLY JSON in this format:
            {
                "optimizations": {
                    "platform": str,
                    "content_changes": list,
                    "timing_changes": list,
                    "format_changes": list,
                    "impact_score": float,
                    "priority": str
                },
                "a_b_test_suggestions": [
                    {
                        "element": str,
                        "variations": list,
                        "hypothesis": str
                    }
                ]
            }
            """,
            llm=platform_manager.model,
            output_type="json"
        )
    
    def generate_optimization_plan(
        self,
        performance_data: AnalyticsReport
    ) -> List[OptimizationSuggestion]:
        try:
            # Prepare performance data for the agent
            analysis_data = {
                "metrics": {
                    platform: {
                        "engagement_rate": metrics.engagement_rate,
                        "reach": metrics.reach,
                        "interactions": metrics.interactions,
                        "raw_metrics": metrics.raw_metrics
                    }
                    for platform, metrics in performance_data.platform_metrics.items()
                },
                "total_performance": {
                    "reach": performance_data.total_reach,
                    "engagement": performance_data.total_engagement
                },
                "time_period": {
                    "start": performance_data.period_start.isoformat(),
                    "end": performance_data.period_end.isoformat()
                }
            }
            
            # Get optimization suggestions
            optimization_response = json.loads(
                self.optimization_agent.run(json.dumps(analysis_data))
            )
            
            # Convert to structured suggestions
            suggestions = []
            for platform, opts in optimization_response["optimizations"].items():
                suggestions.append(
                    OptimizationSuggestion(
                        platform=platform,
                        content_type=opts.get("content_type", "general"),
                        changes=opts["content_changes"] + opts["format_changes"],
                        expected_impact=float(opts["impact_score"]),
                        implementation_priority=opts["priority"]
                    )
                )
            
            return sorted(
                suggestions,
                key=lambda x: x.expected_impact,
                reverse=True
            )
            
        except Exception as e:
            logger.error(f"Optimization plan generation error: {str(e)}")
            raise
    
    async def apply_optimizations(
        self,
        suggestions: List[OptimizationSuggestion],
        content_pieces: Dict[str, ContentPiece]
    ) -> Dict[str, ContentPiece]:
        try:
            optimized_content = content_pieces.copy()
            
            for suggestion in suggestions:
                if suggestion.platform in optimized_content:
                    content = optimized_content[suggestion.platform]
                    
                    # Apply suggested changes
                    for change in suggestion.changes:
                        if "hashtag" in change.lower():
                            # Update hashtags
                            new_hashtags = await self._optimize_hashtags(
                                content.hashtags,
                                suggestion.platform
                            )
                            content.hashtags = new_hashtags
                            
                        elif "timing" in change.lower():
                            # Update posting time
                            new_time = await self._optimize_posting_time(
                                content.best_posting_time,
                                suggestion.platform
                            )
                            content.best_posting_time = new_time
                            
                        elif "format" in change.lower():
                            # Update content format
                            new_format = await self._optimize_format(
                                content.content_type,
                                suggestion.platform
                            )
                            content.content_type = new_format
                    
                    optimized_content[suggestion.platform] = content
            
            return optimized_content
            
        except Exception as e:
            logger.error(f"Optimization application error: {str(e)}")
            raise
    
    async def _optimize_hashtags(
        self,
        current_hashtags: List[str],
        platform: str
    ) -> List[str]:
        try:
            # Get trending hashtags for platform
            if platform == "instagram":
                trending = await self._get_instagram_trending_hashtags()
            elif platform == "linkedin":
                trending = await self._get_linkedin_trending_topics()
            else:
                trending = []
            
            # Combine current and trending hashtags
            all_hashtags = set(current_hashtags + trending)
            
            # Select top performing hashtags
            return list(all_hashtags)[:30]  # Instagram limit
            
        except Exception as e:
            logger.error(f"Hashtag optimization error: {str(e)}")
            raise
    
    async def _optimize_posting_time(
        self,
        current_time: str,
        platform: str
    ) -> str:
        try:
            # Get platform-specific engagement data
            if platform == "linkedin":
                response = self.platform_manager.linkedin_api.get_analytics(
                    time_range="last_30_days",
                    metrics=["engagement_by_time"]
                )
                best_time = self._analyze_engagement_times(response)
                
            elif platform == "instagram":
                insights = self.platform_manager.instagram_account.get_insights(
                    params={
                        'metric': ['engagement_by_time'],
                        'period': 'day'
                    }
                )
                best_time = self._analyze_engagement_times(insights)
                
            else:
                best_time = current_time
            
            return best_time
            
        except Exception as e:
            logger.error(f"Posting time optimization error: {str(e)}")
            raise
    
    async def _optimize_format(
        self,
        current_format: str,
        platform: str
    ) -> str:
        try:
            # Get best performing content formats
            if platform == "instagram":
                insights = self.platform_manager.instagram_account.get_insights(
                    params={
                        'metric': ['engagement_by_format'],
                        'period': 'lifetime'
                    }
                )
                best_format = self._analyze_format_performance(insights)
                
            elif platform == "linkedin":
                analytics = self.platform_manager.linkedin_api.get_analytics(
                    metrics=["engagement_by_format"]
                )
                best_format = self._analyze_format_performance(analytics)
                
            else:
                best_format = current_format
            
            return best_format
            
        except Exception as e:
            logger.error(f"Format optimization error: {str(e)}")
            raise
    
    def _analyze_engagement_times(self, data: Dict[str, Any]) -> str:
        # Implement engagement time analysis logic
        pass
    
    def _analyze_format_performance(self, data: Dict[str, Any]) -> str:
        # Implement format performance analysis logic
        pass

# Example usage
optimizer = ContentOptimizer(platform_manager)

try:
    # Get optimization suggestions
    suggestions = optimizer.generate_optimization_plan(report)
    
    print("Optimization Suggestions:")
    for suggestion in suggestions:
        print(f"Platform: {suggestion.platform}")
        print(f"Priority: {suggestion.implementation_priority}")
        print("Changes:")
        for change in suggestion.changes:
            print(f"- {change}")
        print(f"Expected Impact: {suggestion.expected_impact:.2%}")
    
    # Apply optimizations
    optimized_content = await optimizer.apply_optimizations(
        suggestions,
        content_pieces
    )
    
    print("Optimized Content:")
    for platform, content in optimized_content.items():
        print(f"{platform} Content:")
        print(f"Type: {content.content_type}")
        print(f"Best Time: {content.best_posting_time}")
        print(f"Hashtags: {', '.join(content.hashtags)}")
        
except Exception as e:
    logger.error(f"Optimization workflow error: {str(e)}")
